# <center>人物チェックによるエイムチェックシステム</center>

In [ ]:
import torch
import torchvision
from PIL import Image, ImageDraw
from torchvision import models as models
from torchvision import transforms as transforms
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# モデルを作成する。
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

In [ ]:
# 中心を切り取る関数
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

# 
def detection(file_name):
    # 画像を読み込む
    img = Image.open(file_name).convert('RGB')
    img = crop_center(img,800,600)
    img_tensor = transforms.functional.to_tensor(img).to(device)

    # 推論する。
    output = model([img_tensor])[0]

    # ラベル「人」かつスコアが 0.8 以上の矩形を取り出す。
    PERSON_ID = 1
    cond = (output["labels"] == PERSON_ID) & (output["scores"] >= 0.4)
    person_boxes = output["boxes"][cond].detach().cpu().numpy()
    #print(person_boxes)

    draw = ImageDraw.Draw(img, mode="RGBA")

    high = 0
    wide   = 0
    for box in person_boxes:
        high = (box[0] + box[1]) / 2 
        wide = (box[2] + box[3]) / 2
        draw.rectangle(box, outline="red", width=3)
    print(high,wide)
    print(type(person_boxes))
    print(person_boxes[0])

    plt.imshow(img)
    plt.show()
    
    return

### 画像の出力

In [ ]:
detection("sample.PNG")

In [ ]:
detection("sample1.PNG")